In [1]:
import ast
import os
import csv
import datetime 
from datetime import datetime
import fiona
import requests as r
import numpy as np
import pandas as pd
import geopandas as gp
import glob
from skimage import io
import matplotlib.pyplot as plt
#from osgeo import gdal
import rasterio as rio
import json
from rasterio.mask import mask
from rasterio.enums import Resampling
from rasterio.shutil import copy
from pandas import read_csv
from pathlib import Path
import pickle
import pyproj
from pyproj import Proj
from shapely.ops import transform
import xarray as xr
import geoviews as gv
from cartopy import crs
import hvplot.xarray
import holoviews as hv
gv.extension('bokeh', 'matplotlib')

# Set Up Working Environment
inDir = os.getcwd()
os.chdir(inDir)

In [ ]:
## replace the vi dates with nan values

dir = '.../ALLRUN/nanDataTable'

vinan = read_csv(os.path.join(dir, 'NAweek', 'VInan.csv'))

vimiss = read_csv(os.path.join(dir, 'NAweek', 'VImissingweeklist.csv'))

vinansub = vinan[['Year', 'County', 'Week', 'NDVI', 'TVI', 'EVI', 'SATVI', 'SAVI', 'MSI', 'GNDVI', 'GRVI', 'LSWI', 'TSAVI', 
       'MSAVI', 'MSAVI2', 'WDVI', 'BI', 'BI2', 'RI', 'CI', 'V']]

viname = ['Year', 'County', 'Week', 'NDVI', 'TVI', 'EVI', 'SATVI', 'SAVI', 'MSI', 'GNDVI', 'GRVI', 'LSWI', 'TSAVI', 
       'MSAVI', 'MSAVI2', 'WDVI', 'BI', 'BI2', 'RI', 'CI', 'V']

storevilist = []

for eachline in vimiss.values:

    year = eachline[0]

    county = eachline[1]

    week = eachline[2]

    for eachvalue in vinansub.values:

        if eachvalue[1] == county: 

            if eachvalue[0] == year and eachvalue[2] == week:

                viarray = [year, county, week]

                viarray.extend(['nan'] * 18)
        
                withkey_dict = dict(zip(viname, viarray))

                storevilist.append(withkey_dict)
        
            else: 

                withkey_dict = dict(zip(viname, eachvalue))

                storevilist.append(withkey_dict)
                
keysdict = storevilist[0]

with open(os.path.join(dir, 'NAweek', 'VInan1.csv'), 'w') as f:  
    w = csv.DictWriter(f, keysdict.keys())
    w.writeheader()
    w.writerows(storevilist)        

In [ ]:
###################### calculate the annual means #########################

dir = '.../ALLRUN/nanDataTable'

## wi
prcpdf = read_csv(os.path.join(dir, 'ANNUAL', 'weeklyCR8022.csv'))

gdddf = read_csv(os.path.join(dir, 'ANNUAL', 'weeklyGDD8022.csv'))

## vi
vidf = read_csv(os.path.join(dir, 'ANNUAL', 'VInan.csv'))

## sm
smdf = read_csv(os.path.join(dir, 'ANNUAL', 'SMnan.csv'))

## pr
prdf = read_csv(os.path.join(dir, 'ANNUAL', 'PRnan.csv'))

In [ ]:
df = prdf

UniqueC = df.County.unique()

UniqueY = df.Year.unique()

countylist = []

countyyearlist = []

for elemc in UniqueC: 
    
    countylist.append(df[:][df.County == elemc])

for eachcounty in countylist:

    for elemy in UniqueY: 

        countyyearlist.append(eachcounty[:][eachcounty.Year == elemy])

In [ ]:
avelist = []

for eachitem in countyyearlist:

    year = eachitem.Year.unique()[0]

    county = eachitem.County.unique()[0]

    mean = np.nanmean(eachitem.SM)
            
    withkey_dict = {'Year': year, 'County': county, 'SM': smmean}

    avelist.append(withkey_dict)

keysdict = avelist[0]

with open(os.path.join(dir, 'ANNUAL', 'Average', 'anSM.csv'), 'w') as f:  
    w = csv.DictWriter(f, keysdict.keys())
    w.writeheader()
    w.writerows(avelist)       

In [ ]:
avelist = []

viname = ['Year', 'County', 'NDVI', 'TVI', 'EVI', 'SATVI', 'SAVI', 'MSI', 'GNDVI', 'GRVI', 'LSWI', 'TSAVI', 
       'MSAVI', 'MSAVI2', 'WDVI', 'BI', 'BI2', 'RI', 'CI', 'V']

for eachitem in countyyearlist:

    year = eachitem.Year.unique()[0]

    county = eachitem.County.unique()[0]

    eachitemsub = eachitem[['NDVI', 'TVI', 'EVI', 'SATVI', 'SAVI', 'MSI', 'GNDVI', 'GRVI', 'LSWI', 'TSAVI', 
       'MSAVI', 'MSAVI2', 'WDVI', 'BI', 'BI2', 'RI', 'CI', 'V']]

    viarray = [year, county]

    viarray.extend(np.array(eachitemsub.mean(axis = 'rows', skipna = True)))
        
    withkey_dict = dict(zip(viname, viarray))

    avelist.append(withkey_dict)

keysdict = avelist[0]

with open(os.path.join(dir, 'ANNUAL', 'Average', 'anVI.csv'), 'w') as f:  
    w = csv.DictWriter(f, keysdict.keys())
    w.writeheader()
    w.writerows(avelist)       

In [ ]:
############### all data 80-22 with nan in one csv. file ################

dir = '.../ALLRUN/nanDataTable/ANNUAL/Average'

## prcp
prcpdf = read_csv(os.path.join(dir, 'anCR.csv'))

## gdd 
gdddf = read_csv(os.path.join(dir, 'anGDD.csv'))

## vi
vidf = read_csv(os.path.join(dir, 'anVI.csv'))

## sm
smdf = pd.read_excel(os.path.join(dir, 'anSM.xlsx'))

smdf = smdf.dropna()

## pr
prdf = read_csv(os.path.join(dir, 'anPR.csv'))

## yield
ydf = read_csv(os.path.join(dir, 'Ynan.csv'))


In [ ]:
aver1 = []

for eachline in ydf.values:

    year = eachline[0]

    county = eachline[1]

    newdict = {'Year': year, 'County': county}

    for eachgdd in gdddf.values:

        if eachgdd[0] == year and eachgdd[1] == county:

            inputdict1 = {'GDD': eachgdd[2]}

            aver1.append(dict(newdict, **inputdict1))


In [ ]:
aver2 = []

for eachdict1 in aver1:

    for eachcr in prcpdf.values:
    
        if eachcr[0] == eachdict1['Year'] and eachcr[1] == eachdict1['County']:
    
            inputdict2 = {'CR': eachcr[2]}
                      
            aver2.append(dict(eachdict1, **inputdict2))


In [ ]:
aver3 = []

a = smdf.Year.unique()

b = ydf.Year.unique()

noyear = list((set(a) ^ set(b)))

for eachdict2 in aver2:

    if eachdict2['Year'] in noyear:

        smname = ['SM']
    
        inputdict3 = dict(zip(smname, ['nan'] * 1))
    
        aver3.append(dict(eachdict2, **inputdict3))

    else:

        for eachsm in smdf.values:
    
            if eachsm[0] == eachdict2['Year'] and eachsm[1] == eachdict2['County']:
                
                inputdict3 = {'SM': eachsm[2]}
                                     
                aver3.append(dict(eachdict2, **inputdict3))


In [ ]:
aver4 = []

c = prdf.Year.unique()

b = ydf.Year.unique()

noyear = list((set(c) ^ set(b)))

for eachdict3 in aver3:

    if eachdict3['Year'] in noyear:

        prname = ['PR']
    
        inputdict4 = dict(zip(prname, ['nan'] * 1))
    
        aver4.append(dict(eachdict3, **inputdict4))

    else:

        for eachpr in prdf.values:
    
            if eachpr[0] == eachdict3['Year'] and eachpr[1] == eachdict3['County']:
                
                inputdict4 = {'PR': eachpr[2]}
                                     
                aver4.append(dict(eachdict3, **inputdict4))

In [ ]:
vidf.values[0]

In [ ]:
aver5 = []

d = vidf.Year.unique()

b = ydf.Year.unique()

noyear = list((set(d) ^ set(b)))

for eachdict4 in aver4:

    if eachdict4['Year'] in noyear:

        viname = ['NDVI', 'TVI', 'EVI', 'SATVI', 'SAVI', 'MSI', 'GNDVI', 'GRVI', 'LSWI', 'TSAVI', 
                      'MSAVI', 'MSAVI2', 'WDVI', 'BI', 'BI2', 'RI', 'CI', 'V']

        inputdict5 = dict(zip(viname, ['nan'] * 18))

        aver5.append(dict(eachdict4, **inputdict5))

    else:

        for eachvi in vidf.values:
        
            if eachvi[0] == eachdict4['Year'] and eachvi[1] == eachdict4['County']:
        
                viname = [ 'NDVI', 'TVI', 'EVI', 'SATVI', 'SAVI', 'MSI', 'GNDVI', 'GRVI', 'LSWI', 'TSAVI', 
                          'MSAVI', 'MSAVI2', 'WDVI', 'BI', 'BI2', 'RI', 'CI', 'V']
    
                viarray = eachvi[2:20]
    
                inputdict5 = dict(zip(viname, viarray))
    
                aver5.append(dict(eachdict4, **inputdict5))


In [ ]:
keysdict = aver5[0]

dir1 = '.../ALLRUN/nanDataTable'

with open(os.path.join(dir1, 'ANNUAL', 'ALLnan.csv'), 'w') as f:  
    w = csv.DictWriter(f, keysdict.keys())
    w.writeheader()
    w.writerows(aver5)

In [229]:
################################## weekly data table with nan values in ONE ##################################

## open weekly data table with nan values

dir = '.../ALLRUN/nanDataTable/ANNUAL/Weeklynan'

## prcp
wcrdf = read_csv(os.path.join(dir, 'weeklyCR8022.csv'))

## gdd 
wgdddf = read_csv(os.path.join(dir, 'weeklyGDD8022.csv'))

## sm
wsmdf = pd.read_excel(os.path.join(dir, 'SMnan.xlsx'))

## pr
wprdf = read_csv(os.path.join(dir, 'PRnan.csv'))

## vi
wvidf = read_csv(os.path.join(dir, 'VInan.csv'))

In [140]:
wprdf

,Year,County,Week,PR
0,2015,ADAMS,20.0,0.129372
1,2015,ADAMS,21.0,0.191616
2,2015,ADAMS,22.0,0.224887
3,2015,ADAMS,23.0,0.253716
4,2015,ADAMS,24.0,0.270219
...,...,...,...,...
19963,2022,WOODFORD,41.0,0.922339
19964,2022,WOODFORD,42.0,0.954620
19965,2022,WOODFORD,43.0,0.968694
19966,2022,WOODFORD,44.0,0.988228


In [ ]:
### create a data table containing 80-22 * 26 weeks

Yield96df = read_csv('.../ALLRUN/Y/182296Yield.csv')

Headarray = ['Year', 'County', 'Week']

Headdict = {n: n for n in Headarray} 

newlist = []

for year in [str(x) for x in list(range(1980, 2023))]:

    for elem in Yield96df.County.unique():
        
        for x in range(20, 46):

            newlist.append(np.hstack((year, elem, x)))

    withkey_dict = []

    for each in newlist:

        withkey_dict.append(dict(zip(Headarray, each)))
    
with open(os.path.join(dir, '8022coun96Week2045' + '.csv'), 'w') as f:  
    w = csv.DictWriter(f, Headdict.keys())
    w.writeheader()
    w.writerows(withkey_dict)

In [3]:
### open the full week data frame and fill the weekly values to it ###

empty = read_csv(os.path.join(dir, '8022coun96Week2045' + '.csv'))

In [4]:
newlist = []

for eachline in empty.values:

    year = eachline[0]

    county = eachline[1]

    week = eachline[2]

    Headdict = {'Year': year, 'County': county, 'Week': week}

    for eachwcr in wcrdf.values:

        if eachwcr[0] == year and eachwcr[1] == county and eachwcr[2] == week:

            wcrdict = {'CR': eachwcr[3]}

            newdict1 = dict(Headdict, **wcrdict)

    for eachwgdd in wgdddf.values:

        if eachwgdd[0] == year and eachwgdd[1] == county and eachwgdd[2] == week:

            wgdddict = {'GDD': eachwgdd[3]}

            newdict2 = dict(newdict1, **wgdddict)

            newlist.append(newdict2)


In [112]:
### plus sm

plussm = []

for eachdict in newlist:

    for eachwsm in wsmdf.values:
    
        if eachwsm[0] == eachdict['Year'] and eachwsm[1] == eachdict['County'] and eachwsm[2] == eachdict['Week']:
    
           wsmdict = {'SM': eachwsm[3]}
    
           plussm.append(dict(eachdict, **wsmdict)) 

In [131]:
a = wsmdf.Year.unique()

b = wcrdf.Year.unique()

noyear = list((set(a) ^ set(b)))

noyearlist = [d for d in newlist if d['Year'] in noyear]

nansmlist = []

for eachdict in noyearlist:

    smname = ['SM']
    
    nansmdict = dict(zip(smname, ['nan'] * 1))
    
    nansmlist.append(dict(eachdict, **nansmdict))

In [137]:
fullwsm = nansmlist + plussm

In [142]:
fullwsm

[{'Year': 1980,
  'County': 'ADAMS',
  'Week': 20,
  'CR': 62.36737654,
  'GDD': 38.15319710015261,
  'SM': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 21,
  'CR': 7.7082777,
  'GDD': 62.18413893948946,
  'SM': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 22,
  'CR': 48.282264260000005,
  'GDD': 94.9262718667628,
  'SM': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 23,
  'CR': 39.837212,
  'GDD': 91.85162232584972,
  'SM': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 24,
  'CR': 8.86561798,
  'GDD': 79.83114926745783,
  'SM': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 25,
  'CR': 0.31685063,
  'GDD': 65.96939627184412,
  'SM': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 26,
  'CR': 8.140187,
  'GDD': 110.8242220621386,
  'SM': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 27,
  'CR': 49.74205979999999,
  'GDD': 109.01425514221192,
  'SM': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 28,
  'CR': 0.3688962

In [147]:
## plus pr

pluspr = []

for eachdict in fullwsm:

    for eachwpr in wprdf.values:
    
        if eachwpr[0] == eachdict['Year'] and eachwpr[1] == eachdict['County'] and eachwpr[2] == eachdict['Week']:
    
           wprdict = {'PR': eachwpr[3]}
    
           pluspr.append(dict(eachdict, **wprdict)) 

In [148]:
pluspr

[{'Year': 2015,
  'County': 'ADAMS',
  'Week': 20,
  'CR': 20.63505449,
  'GDD': 52.707024930028005,
  'SM': 0.0062557478,
  'PR': 0.12937234},
 {'Year': 2015,
  'County': 'ADAMS',
  'Week': 21,
  'CR': 34.1518328,
  'GDD': 35.960296649458,
  'SM': 0.25402505,
  'PR': 0.19161588},
 {'Year': 2015,
  'County': 'ADAMS',
  'Week': 22,
  'CR': 32.962738737399995,
  'GDD': 69.39627388324975,
  'SM': 0.2701290466666666,
  'PR': 0.22488695},
 {'Year': 2015,
  'County': 'ADAMS',
  'Week': 23,
  'CR': 27.9469846,
  'GDD': 67.07326771233588,
  'SM': 0.1795485265666666,
  'PR': 0.25371638},
 {'Year': 2015,
  'County': 'ADAMS',
  'Week': 24,
  'CR': 50.087212,
  'GDD': 103.20684852520954,
  'SM': 0.2954018233333333,
  'PR': 0.2702191},
 {'Year': 2015,
  'County': 'ADAMS',
  'Week': 25,
  'CR': 90.156924,
  'GDD': 92.41852484738678,
  'SM': 0.234038935,
  'PR': 0.27809268},
 {'Year': 2015,
  'County': 'ADAMS',
  'Week': 26,
  'CR': 109.50682618,
  'GDD': 87.47428837376513,
  'SM': 0.35181555,
  'PR'

In [149]:
c = wprdf.Year.unique()

b = wcrdf.Year.unique()

noyear1 = list((set(c) ^ set(b)))

noyearlist1 = [d for d in fullwsm if d['Year'] in noyear1]

In [150]:
noyearlist1

[{'Year': 1980,
  'County': 'ADAMS',
  'Week': 20,
  'CR': 62.36737654,
  'GDD': 38.15319710015261,
  'SM': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 21,
  'CR': 7.7082777,
  'GDD': 62.18413893948946,
  'SM': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 22,
  'CR': 48.282264260000005,
  'GDD': 94.9262718667628,
  'SM': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 23,
  'CR': 39.837212,
  'GDD': 91.85162232584972,
  'SM': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 24,
  'CR': 8.86561798,
  'GDD': 79.83114926745783,
  'SM': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 25,
  'CR': 0.31685063,
  'GDD': 65.96939627184412,
  'SM': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 26,
  'CR': 8.140187,
  'GDD': 110.8242220621386,
  'SM': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 27,
  'CR': 49.74205979999999,
  'GDD': 109.01425514221192,
  'SM': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 28,
  'CR': 0.3688962

In [151]:
nanprlist = []

for eachdict in noyearlist1:

    prname = ['PR']
    
    nanprdict = dict(zip(prname, ['nan'] * 1))
    
    nanprlist.append(dict(eachdict, **nanprdict))

In [152]:
nanprlist

[{'Year': 1980,
  'County': 'ADAMS',
  'Week': 20,
  'CR': 62.36737654,
  'GDD': 38.15319710015261,
  'SM': 'nan',
  'PR': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 21,
  'CR': 7.7082777,
  'GDD': 62.18413893948946,
  'SM': 'nan',
  'PR': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 22,
  'CR': 48.282264260000005,
  'GDD': 94.9262718667628,
  'SM': 'nan',
  'PR': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 23,
  'CR': 39.837212,
  'GDD': 91.85162232584972,
  'SM': 'nan',
  'PR': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 24,
  'CR': 8.86561798,
  'GDD': 79.83114926745783,
  'SM': 'nan',
  'PR': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 25,
  'CR': 0.31685063,
  'GDD': 65.96939627184412,
  'SM': 'nan',
  'PR': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 26,
  'CR': 8.140187,
  'GDD': 110.8242220621386,
  'SM': 'nan',
  'PR': 'nan'},
 {'Year': 1980,
  'County': 'ADAMS',
  'Week': 27,
  'CR': 49.74205979999999,
  'GDD':

In [153]:
fullwpr = nanprlist + pluspr

In [155]:
len(fullwpr)

107328

In [232]:
wvidf = wvidf[['Year', 'County', 'Week', 'NDVI', 'TVI', 'EVI', 'SATVI', 'SAVI', 'MSI', 'GNDVI', 'GRVI', 'LSWI', 'TSAVI', 
                          'MSAVI', 'MSAVI2', 'WDVI', 'CI', 'V']]

In [233]:
wvidf

,Year,County,Week,NDVI,TVI,EVI,SATVI,SAVI,MSI,GNDVI,GRVI,LSWI,TSAVI,MSAVI,MSAVI2,WDVI,CI,V
0,2018,ADAMS,20,0.698585,109.177939,0.475768,0.174176,0.530209,0.652639,0.666868,0.115683,0.233527,1.320925,0.468204,0.469704,0.278244,-0.115683,14.969722
1,2018,ADAMS,21,0.576137,103.224633,0.402827,0.184228,0.454853,0.930984,0.585288,0.038701,0.082898,0.967914,0.376418,0.377204,0.226014,-0.038701,6.368340
2,2018,ADAMS,22,0.615334,105.188960,0.458570,0.204124,0.512928,0.874411,0.612084,0.049976,0.101031,1.138225,0.428455,0.429768,0.264022,-0.049976,6.439249
3,2018,ADAMS,23,0.615237,105.143381,0.472214,0.191537,0.519488,0.835052,0.611940,0.057139,0.127002,1.177220,0.440244,0.438902,0.270841,-0.057139,7.149718
4,2018,ADAMS,24,0.682492,108.558717,0.530050,0.176216,0.572839,0.629841,0.663129,0.052709,0.246294,1.389260,0.496353,0.496837,0.312213,-0.052709,6.339360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12475,2022,WOODFORD,41,0.474049,98.164326,0.271500,0.105037,0.366318,0.818130,0.576627,-0.120775,0.118186,0.561394,0.263792,0.269424,0.161765,0.120775,3.743877
12476,2022,WOODFORD,42,0.374929,93.209995,0.233082,0.117251,0.344015,0.987754,0.477081,-0.119507,0.020423,0.378249,0.222271,0.227672,0.144509,0.119507,2.461953
12477,2022,WOODFORD,43,0.397754,94.475624,0.251421,0.104749,0.366122,0.914807,0.554102,-0.188541,0.058544,0.493450,0.252317,0.255683,0.163872,0.188541,2.931768
12478,2022,WOODFORD,44,0.359507,92.350305,0.199184,0.138236,0.286279,1.217282,0.492030,-0.154352,-0.085682,0.287929,0.187282,0.191810,0.115674,0.154352,2.435868


In [234]:
wvidf.values[0][3:19]

array([0.698584787052241, 109.177938814659, 0.475767750602348,
       0.174175515816525, 0.530209436957196, 0.652638882085388,
       0.666868287469362, 0.11568332872164, 0.233526509039364,
       1.32092451705824, 0.468204078931954, 0.469703995632363, 0.27824368,
       -0.11568332872164, 14.9697217976019], dtype=object)

In [235]:
## plus vi

plusvi = []

for eachdict in fullwpr:

    for eachwvi in wvidf.values:
    
        if eachwvi[0] == eachdict['Year'] and eachwvi[1] == eachdict['County'] and eachwvi[2] == eachdict['Week']:
    
           viname = ['NDVI', 'TVI', 'EVI', 'SATVI', 'SAVI', 'MSI', 'GNDVI', 'GRVI', 'LSWI', 'TSAVI', 
                          'MSAVI', 'MSAVI2', 'WDVI', 'CI', 'V']
    
           viarray = eachwvi[3:19]
            
           wvidict = dict(zip(viname, viarray))
    
           plusvi.append(dict(eachdict, **wvidict)) 

In [236]:
plusvi

[{'Year': 2018,
  'County': 'ADAMS',
  'Week': 20,
  'CR': 20.033361,
  'GDD': 80.10448695619053,
  'SM': 0.1686882499999999,
  'PR': 0.2599614,
  'NDVI': 0.698584787052241,
  'TVI': 109.177938814659,
  'EVI': 0.475767750602348,
  'SATVI': 0.174175515816525,
  'SAVI': 0.530209436957196,
  'MSI': 0.652638882085388,
  'GNDVI': 0.666868287469362,
  'GRVI': 0.11568332872164,
  'LSWI': 0.233526509039364,
  'TSAVI': 1.32092451705824,
  'MSAVI': 0.468204078931954,
  'MSAVI2': 0.469703995632363,
  'WDVI': 0.27824368,
  'CI': -0.11568332872164,
  'V': 14.9697217976019},
 {'Year': 2018,
  'County': 'ADAMS',
  'Week': 21,
  'CR': 15.503486300000002,
  'GDD': 99.43123002762096,
  'SM': 0.1318000383333333,
  'PR': 0.2951114,
  'NDVI': 0.576137421066312,
  'TVI': 103.224633474725,
  'EVI': 0.402827486098086,
  'SATVI': 0.184227968622769,
  'SAVI': 0.454853236194206,
  'MSI': 0.930984149637904,
  'GNDVI': 0.585288341375998,
  'GRVI': 0.0387008488204341,
  'LSWI': 0.0828978448853928,
  'TSAVI': 0.9679

In [237]:
d = wvidf.Year.unique()

b = wcrdf.Year.unique()

noyear2 = list((set(d) ^ set(b)))

noyearlist2 = [d for d in fullwpr if d['Year'] in noyear2]

nanvilist = []

for eachdict in noyearlist2:

    viname = ['NDVI', 'TVI', 'EVI', 'SATVI', 'SAVI', 'MSI', 'GNDVI', 'GRVI', 'LSWI', 'TSAVI', 
                          'MSAVI', 'MSAVI2', 'WDVI', 'CI', 'V']
    
    nanvidict = dict(zip(viname, ['nan'] * 15))
    
    nanvilist.append(dict(eachdict, **nanvidict))

In [238]:
fullwvi = nanvilist + plusvi

len(fullwvi)

107328

In [174]:
## plus srad

dir = '.../ALLRUN/nanDataTable/ANNUAL/Weeklynan'

## srad
wsrdf = read_csv(os.path.join(dir, 'weeklySR8021.csv'))

In [195]:
wsrdf

,Year,County,Week,srad
0,1980,ADAMS,20,2237.868620
1,1980,ADAMS,21,2662.558960
2,1980,ADAMS,22,2573.622510
3,1980,ADAMS,23,2497.703010
4,1980,ADAMS,24,3130.090640
...,...,...,...,...
104827,2021,WOODFORD,41,1520.859225
104828,2021,WOODFORD,42,1665.888420
104829,2021,WOODFORD,43,1321.713125
104830,2021,WOODFORD,44,2035.757190


In [239]:
len(fullwvi)

107328

In [243]:
plussr = []

for eachdict in fullwvi:

    for eachwsr in wsrdf.values:
    
        if eachwsr[0] == eachdict['Year'] and eachwsr[1] == eachdict['County'] and eachwsr[2] == eachdict['Week']:
    
           wsrdict = {'SR': eachwsr[3]}
    
           plussr.append(dict(eachdict, **wsrdict)) 

In [245]:
e = wsrdf.Year.unique()

b = wcrdf.Year.unique()

noyear3 = list((set(e) ^ set(b)))

noyearlist3 = [e for e in fullwvi if e['Year'] in noyear3]

nansrlist = []

for eachdict in noyearlist3:

    srname = ['SR']
    
    nansrdict = dict(zip(srname, ['nan'] * 1))
    
    nansrlist.append(dict(eachdict, **nansrdict))

In [246]:
fullwsr = plussr + nansrlist

len(fullwsr)

107328

In [247]:
## plus vp

dir = '.../ALLRUN/nanDataTable/ANNUAL/Weeklynan'

## vp
wvpdf = read_csv(os.path.join(dir, 'weeklyVP8021.csv'))

In [251]:
plusvp = []

for eachdict in fullwsr:

    for eachwvp in wvpdf.values:
    
        if eachwvp[0] == eachdict['Year'] and eachwvp[1] == eachdict['County'] and eachwvp[2] == eachdict['Week']:
    
           wvpdict = {'VP': eachwvp[3]}
    
           plusvp.append(dict(eachdict, **wvpdict)) 

In [254]:
f = wvpdf.Year.unique()

b = wcrdf.Year.unique()

noyear4 = list((set(f) ^ set(b)))

noyearlist4 = [f for f in fullwsr if f['Year'] in noyear4]

nanvplist = []

for eachdict in noyearlist4:

    vpname = ['VP']
    
    nanvpdict = dict(zip(vpname, ['nan'] * 1))
    
    nanvplist.append(dict(eachdict, **nanvpdict))


In [255]:
fullw = plusvp + nanvplist

len(fullw)

107328

In [256]:
keysdict = fullw[0]

dir1 = '.../ALLRUN/nanDataTable/ANNUAL'

with open(os.path.join(dir1, 'WeeklyNA.csv'), 'w') as f:  
    w = csv.DictWriter(f, keysdict.keys())
    w.writeheader()
    w.writerows(fullw)  